In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import json
import cv2
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



Create Dataset

In [4]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
    def __len__(self):
        return len(self.data)
    
    # Takes index and returns the image and keypoints at that index
    def __getitem__(self, index):
        item = self.data[index]
        #load image
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        #get height and width
        height, width = img.shape[:2]
        # Change format to rgb else "cpu"
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Transform image to predefined transformations
        img = self.transforms(img)
        # Array of keypoint loacations and convert it to a 1d array
        kps = np.array(item['kps']).flatten()
        #Make array elements floats
        kps = kps.astype(np.float32)
        # Adjust keypoint locations with new image size
        kps[::2] *= 224.0/width
        kps[1::2] *= 224.0/height
        return img, kps

In [5]:
train_dataset = KeypointsDataset("./data/images", "data/data_train.json")
val_dataset = KeypointsDataset("./data/images", "./data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

Create Model

In [6]:
model = models.resnet50(pretrained = True)
# Replaces last layer of the network and makes it linear with 28 output nodes
model.fc = torch.nn.Linear(model.fc.in_features, 14*2)

model = model.to(device)

c:\Users\dagmh\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\dagmh\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Train Model

In [7]:
# Define loss function
crit = torch.nn.MSELoss()
#Define Optimizer function
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

In [9]:
epochs = 20
#loop over number of epochs
for epoch in range(epochs):
    # Loop over image and keypoints
    for i, (imgs, kps) in enumerate(train_loader):
        #Move info to device
        imgs = imgs.to(device)
        kps = kps.to(device)

        #Flush gradients from the optimizer
        optimizer.zero_grad()
        # Predict keypoints in images provided
        pred = model(imgs)
        #Calculate loss
        loss = crit(pred, kps)  
        #Back prop
        loss.backward()
        #Take learning step
        optimizer.step()
        #Log every 10 steps
        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14547.03125
Epoch 0, iter 10, loss: 14602.6640625
Epoch 0, iter 20, loss: 14159.439453125
Epoch 0, iter 30, loss: 14141.98046875
Epoch 0, iter 40, loss: 13789.3388671875
Epoch 0, iter 50, loss: 13267.2373046875
Epoch 0, iter 60, loss: 12826.8662109375
Epoch 0, iter 70, loss: 12251.7744140625
Epoch 0, iter 80, loss: 11058.8955078125
Epoch 0, iter 90, loss: 12141.07421875
Epoch 0, iter 100, loss: 11508.04296875
Epoch 0, iter 110, loss: 10553.9443359375
Epoch 0, iter 120, loss: 10402.7021484375
Epoch 0, iter 130, loss: 9979.05859375
Epoch 0, iter 140, loss: 10453.0810546875
Epoch 0, iter 150, loss: 9952.44140625
Epoch 0, iter 160, loss: 8893.5390625
Epoch 0, iter 170, loss: 8634.0380859375
Epoch 0, iter 180, loss: 8282.5078125
Epoch 0, iter 190, loss: 8289.6640625
Epoch 0, iter 200, loss: 7989.53173828125
Epoch 0, iter 210, loss: 8385.2763671875
Epoch 0, iter 220, loss: 6727.71728515625
Epoch 0, iter 230, loss: 6337.16162109375
Epoch 0, iter 240, loss: 7290.84765625

In [10]:
#Save model to file
torch.save(model.state_dict(), "Models/keypoints_model.pth")